## Collaborative Filtering - Item based ## 

#### Most popular method behind recommendation systems ####

Recommendation systems are broadly divided into two types
1. Recommend me items based on what my peers have liked in the past - Collaborative filtering
2. Recommend me items based on what I've liked in the past - Content-based filtering

In this notebook, we will implement Collaborative Filtering using Python on Movie Lens Dataset

Let me describe, what would be my steps like:

*Inputs*
1. User profile: Purchase history of user, preferences
2. Item profile: Characteristics of item. for e.g. movie's genre, lead actor, etc
3. Rating matrix: a table of user's rating for each item. this would be a sparse matrix

Steps:
- For each item not yet seen by Alice 
    - Find similar users to Alice who've rated this item
        - Find the average of similar user's rating for the item

In [142]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys

In [143]:
users = pd.read_csv('data/ml-100k/u.user', sep='|', encoding='latin', header=None,
                    names=["user id","age","gender","occupation","zip code"])

In [44]:
items = pd.read_csv('data/ml-100k/u.item', sep='|', encoding='latin', header=None,
                    names=["movie id" , "movie title" , "release date" , "video release date" , "IMDb URL" , "unknown" , "Action" , "Adventure" , "Animation" , 
                               "Children's" , "Comedy" , "Crime" , "Documentary" , "Drama" , "Fantasy" , 
                               "Film-Noir" , "Horror" , "Musical" , "Mystery" , "Romance" , "Sci-Fi" , "Thriller" , "War" , "Western"])

In [45]:
items.columns

Index(['movie id', 'movie title', 'release date', 'video release date',
       'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children's',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [46]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Rating matrix

In [37]:
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t',encoding='latin', 
                    names=["user id" , "item id" , "rating" , "timestamp"])

In [38]:
ratings.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [21]:
ratings_train = ratings[:int(ratings.shape[0]*0.8)]
ratings_test = ratings[int(ratings.shape[0]*0.8):]

In [22]:
ratings_test.shape

(20000, 4)

In [23]:
ratings_train.shape

(80000, 4)

In [47]:
unique_items = items['movie id'].unique()

In [144]:
unique_users = users['user id'].unique()

In [ ]:
similarity_matrix = np.zeros(dtype=float, shape= (len(unique_items)+1, len(unique_items)+1))
for itemA in unique_items:   
    print('Finding similar items for ',itemA)
    for itemB in unique_items:        
        if itemB != itemA:
            # calculate similarity
            
            itemAUsers = ratings_train[ratings_train['item id']==itemA]['user id'].unique()
            itemBUsers = ratings_train[ratings_train['item id']==itemB]['user id'].unique()
            
            commonUsers = np.intersect1d(itemAUsers,itemBUsers)
            #print('itemA {}, itemB {}, commonUsers - {}'.format(itemA, itemB, commonUsers)) 
            
            similarity_matrix[itemA, itemB] = similarity_matrix[itemB, itemA] = calculate_pearson_similarity((itemA, itemB), commonUsers, ratings)
                                        
        else:
            continue

The above logic takes n^2 time where n is number of items. Instead, we can look at product catalog and see the pairs that actually exists

Find pairs by going through basket of every user
For e.g. 

User1 = {1,2,4,5}

User2 = {2,6,3,8}

Pairs = { (1,2), (1,4), (1,5), (2,6), (2,3), (2,8) }

In [196]:
pairs = []

def cartesian_product_of_list(elements):
    
    pairs = [(elements[i],elements[j]) for i in range(0,len(elements)) for j in range(i+1, len(elements)) ]
    return pairs

items_purchased_by_user = ratings.groupby('user id')['item id'].apply(tuple).apply(cartesian_product_of_list)      


pairs = np.concatenate(items_purchased_by_user.values)

unique_pairs = np.unique(pairs, axis=0)

len(unique_pairs)

In [218]:
similarity_matrix = np.zeros(dtype=float, shape= (len(unique_items)+1, len(unique_items)+1))
for itemA, itemB in unique_pairs:   
#     print('Finding similar items for ',itemA)      
    if itemB != itemA:
        # calculate similarity

        itemAUsers = ratings_train[ratings_train['item id']==itemA]['user id'].unique()
        itemBUsers = ratings_train[ratings_train['item id']==itemB]['user id'].unique()

        commonUsers = np.intersect1d(itemAUsers,itemBUsers)
        #print('itemA {}, itemB {}, commonUsers - {}'.format(itemA, itemB, commonUsers)) 

        similarity_matrix[itemA, itemB] = similarity_matrix[itemB, itemA] = calculate_pearson_similarity((itemA, itemB), commonUsers, ratings)

    else:
        continue

KeyboardInterrupt: 

In [152]:
similarity_matrix[4,1000]

0.0

In [120]:
#Test Similarity function

items = (1,2)
test_common_users = [  1,5,13,64,72,83,92,95,130,178,193,200,213,222,249,250,256,268
,271,276,279,280,292,293,301,303,305,320,325,327,363,374,378,379,387,407
,416,425,429,435,450,455,472,484,487,497,561,618,621,622,642,648,655,660
,705,715,727,746,749,751,773,796,830,864,916]

calculate_pearson_similarity(items, test_common_users, ratings)

-0.0871681159948


In [125]:
def calculate_pearson_similarity(items, commonUsers, ratings):
    
    numerator = 0
    
    sumOfSqrdDiffA = 0
    
    sumOfSqrdDiffB = 0
    
    for commonUser in commonUsers:
        itemA = items[0]
        itemB = items[1]
        avgUserRating = ratings[ratings['user id']==commonUser]['rating'].mean()


        ratingADistanceFromAvgRating = ratings[(ratings['user id']==commonUser) & (ratings['item id']==itemA)]['rating'].item()-avgUserRating

        ratingADistanceFromAvgRatingSqrd = ratingADistanceFromAvgRating * ratingADistanceFromAvgRating
        
        ratingBDistanceFromAvgRating = ratings[(ratings['user id']==commonUser) & (ratings['item id']==itemB)]['rating'].item()-avgUserRating
        
        ratingBDistanceFromAvgRatingSqrd = ratingBDistanceFromAvgRating * ratingBDistanceFromAvgRating
        
        numerator += ratingADistanceFromAvgRating * ratingBDistanceFromAvgRating
        
        sumOfSqrdDiffA += ratingADistanceFromAvgRatingSqrd 
        
        sumOfSqrdDiffB += ratingBDistanceFromAvgRatingSqrd
        
    denominator = np.sqrt(sumOfSqrdDiffA) * np.sqrt(sumOfSqrdDiffB)
        
    if denominator != 0:
        return numerator / denominator